In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Concatenate,Dropout

model = Sequential()

# Add a 3D convolutional layer
model.add(Conv3D(filters=16, kernel_size=(3, 3, 3), strides=(2, 2, 2), activation='relu', input_shape=(155, 240, 240, 4)))
model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(2, 2, 2), activation='relu'))
model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), strides=(2, 2, 2), activation='relu'))
model.add(Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(2, 2, 2), activation='relu'))
# Here, (240, 240, 155, 4) specifies the input shape: (depth, height, width, channels)
# The kernel_size is (3, 3, 3) with 32 filters

# Flatten the output from the convolutional layers
model.add(Flatten())
model.add(Dropout(0.2))
# Add fully connected (Dense) layers

model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=1, activation='relu'))
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# Choose optimizer, loss function, and metrics as needed

# Print a summary of the model architecture
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_50 (Conv3D)          (None, 77, 119, 119, 16   1744      
                             )                                   
                                                                 
 conv3d_51 (Conv3D)          (None, 38, 59, 59, 32)    13856     
                                                                 
 conv3d_52 (Conv3D)          (None, 18, 29, 29, 64)    55360     
                                                                 
 conv3d_53 (Conv3D)          (None, 8, 14, 14, 128)    221312    
                                                                 
 flatten_14 (Flatten)        (None, 200704)            0         
                                                                 
 dropout_1 (Dropout)         (None, 200704)            0         
                                                     

In [41]:
import pandas as pd
import os
from skimage.transform import resize
from sklearn.utils.validation import column_or_1d
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor

flat_data_arr=[] #input array
target_arr=[] #output array

#file path
path = 'C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/myData'
surInfoFile = "C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data/survival_info.csv"

date_column = ['Brats20ID','Age','Survival_days','Extent_of_Resection']
df_surInfo = pd.read_csv(surInfoFile,names= date_column,skiprows=1)


def modify_value(value):
    idStr = str(value)
    number_str = idStr[-3:] 
    number_int = int(number_str)
    return number_int

    
df_surInfo['Brats20ID'] = df_surInfo['Brats20ID'].apply(modify_value)
print(df_surInfo)

     Brats20ID     Age  Survival_days Extent_of_Resection
0            1  60.463            289                 GTR
1            2  52.263            616                 GTR
2            3  54.301            464                 GTR
3            4  39.068            788                 GTR
4            5  68.493            465                 GTR
..         ...     ...            ...                 ...
230        363  57.000             62                 GTR
231        366  72.000            633                 GTR
232        367  60.000            437                 STR
233        368  49.000            442                 GTR
234        369  50.000            145                 GTR

[235 rows x 4 columns]


In [42]:
pattern = r"volume_(\d+)_"
pattern2 = r"slice_(\d+)"

y = pd.DataFrame(columns = ['Survival_days','Volume'])
volumes = []
#path which contains all the categories of images
fileCount = 0
volumeCount = 0
volumeCountMax = 20

volumeId = -1
train_num = 200
count = 0

MSE = 0.0
dataCount = 0
dataVolume = []
for img in os.listdir(path):
    
    
    if img.endswith(".h5"):
        count += 1
        
        
        fileName = os.path.join(path,img)
        dataId = re.search(pattern, fileName).group(1)
        dataId = int(dataId)
        SliceId = re.search(pattern2, fileName).group(1)
        SliceId = int(SliceId)

        if volumeId != dataId :
            volumeId = dataId
            volumeCount += 1
        
        if dataId in df_surInfo['Brats20ID'].values:
    

            #print(f'dataId :{dataId}')
            #print(f'SliceId :{SliceId}')
            f = h5py.File(fileName, 'r')
            img_array = f.get('data')
            img_array = np.array(img_array)
            dataVolume.append(img_array)
            
            fileCount += 1
                
            if fileCount == 155:
                survDays = df_surInfo.loc[df_surInfo['Brats20ID'] == dataId, 'Survival_days'].iloc[0]
                Y = np.array([survDays])
                print(Y)
                dataVolume = np.array([dataVolume])
                print(dataVolume.shape)
                if(volumeCount <= train_num):
                
                    print('Training')
                    loss, mae = model.train_on_batch(dataVolume, Y)
                    print('The Model is trained well with the given images')
                    print('loss:' + str(loss))
                    print('m:' + str(mae))
                
                else:
                    print('Testing')
                    y_pred=model.predict_on_batch(np.array(dataVolume))
                    print("The predicted Data is :")
                    print(y_pred)
                    print("The actual data is:")
                    print(np.array(Y))
                    mse = mean_squared_error(y_pred,Y)
                    MSE += mse
                    dataCount += 1
                    print(f"The model MSE is {mse}")

                dataVolume = []
                fileCount = 0

            
                
                
            
print(f"dataCount : {dataCount}")
print(f"The model MSE is {MSE/dataCount}")


[277]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:76618.0
m:0.0
[155]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:9225.544921875
m:0.0
[1148]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:412910.25
m:0.0
[78]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:4206818.5
m:0.0
[580]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:45098.265625
m:0.0
[1145]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:752925.875
m:0.0
[77]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:3740.385498046875
m:0.0
[427]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:125794.28125
m:0.0
[409]
(1, 155, 240, 240, 4)
Training
The Model is trained well with the given images
loss:133032.75
m:0.0
[112]
(1, 155, 240, 240, 4)
